# Example: Using Linear Programming to Find Shortest Paths Through a Metabolic Network
In this example, we'll use Linear Programming to compute the shortest paths in a metabolic network. This network connects a `source` compound, the starting point of our metabolic pathway, and a `target` compound, the endpoint. Computing these paths is crucial in metabolic engineering and related fields. To illustrate this, we'll use a simple metabolic network adapted from a [paper published by Palsson and coworkers](https://www.sciencedirect.com/science/article/pii/S0006349503748991?via%3Dihub).

<!-- ![alt text](figs/ThreeGene-Network.png "Title") -->
<center>
    <img src="figs/Fig-ToyNetworkSchematic.svg" width="480" height="336">
</center>

In this toy network, chemical transformations occur inside a physical or logical control volume, where each chemical species is a node in the graph above. Each edge in the graph corresponds to a chemical reaction or an exchange mechanism. 
* The chemical reaction system is assumed to be in a steady state inside the control volume. The chemical species (nodes) interact with one another through reactions (edges) labeled as `R` in the schematic above.
* The control volume exchanges metabolites with the surroundings through logical or physical transfer mechanisms called exchanges, which are edges labeled as `E` in the schematic above.

## Setup
The computations in this lab (or example) are enabled by codes in the [src](src) directory and several external `Julia` packages. To load the required packages and any custom codes the teaching team has developed to work with these packages, we [include](https://docs.julialang.org/en/v1/manual/code-loading/) the `Include.jl` file):

In [3]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/LP-Metabolic-Flux-Paths `
  No Changes to `~/Desktop/julia_work/LP-Metabolic-Flux-Paths /Project.toml`
  No Changes to `~/Desktop/julia_work/LP-Metabolic-Flux-Paths /Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/julia_work/LP-Metabolic-Flux-Paths /Project.toml`
  No Changes to `~/Desktop/julia_work/LP-Metabolic-Flux-Paths /Manifest.toml`


## Prerequisites
Before we compute the shortest path, let's load the metabolic network and create an instance of the `MyStoichiometricMatrixModel`, which holds data about the reaction network defined in [Types.jl](src/Types.jl). We construct a `MyStoichiometricMatrixModel` instance using a `build` method defined in [Factory.jl](src/Factory.jl). First, let's set the path to the `network` file, and save it in the `path_to_model_file`:

In [5]:
path_to_model_file = joinpath(_PATH_TO_DATA, "Network-1.net");

Next, let's read the `network` file using the `readreactionfile(...)` method, which is encoded in [Files.jl](src/Files.jl). This method takes the path to the `network` file and the `expand::Bool` flag, which tells the parser whether or not to split reversible reactions into separate forward and reverse rates. 
* The `readreactionfile(...)` method returns a `Dict{Int64, MyChemicalReactionModel}` where the `key` is the reaction index, and the `value` is a `MyChemicalReactionModel` model holding the data for a reaction.
* We pass the `edges` dictionary to a `build(...)` method which returns an instance of `MyStoichiometricMatrixModel` which we store in the `model` variable.

In [52]:
model = readreactionfile(path_to_model_file, expand=true) |> edges-> build(MyStoichiometricMatrixModel, edges);

## Compute the minimum cost path through the network
Let's assume each reaction (edge) in the network has a cost of `1`. Then, to compute the minimum cost path that connects a `source` metabolite to a `target` metabolite, we solve a linear programming problem. Let $\mathcal{O}(\mathbf{x})$ denote a linear function of the continuous non-negative decision variables $\mathbf{v}\in\mathbb{R}^{m}$, i.e., the flux through the edges in the network, whose values are constrained by a system of linear equations and bounded. 
Then, the optimal lowest cost path $\mathbf{v}^{\star}$ is a solution of the linear program:
\begin{eqnarray*}
\text{minimize}~\mathcal{O}(\mathbf{x}) &=& \sum_{i=1}^{m} c_{i}\cdot{v}_{i}\\
\text{subject to}~\mathbf{S}\cdot\mathbf{v} & = &\mathbf{0}\\
\text{and}~L_{i}&\leq{v_{i}}&\leq{U_{i}}\qquad{i=1,2,\dots,m}
\end{eqnarray*}
The constants $c_{i} = 1$ are coefficients in the objective function, $\mathbf{S}\in\mathbb{R}^{n\times{m}}$ is the stoichiometric matrix,
and $\mathbf{S}\cdot\mathbf{v} = \mathbf{0}$ occurs because the reaction system is constrained to be at a steady state inside the logical or physical control volume. The lower ($L_{i}$) and upper ($U_{i}$) bound for each reaction is what we can use to specify the `source` and `target` metabolites.
* To specify `source` and `target` metabolites, we specify the permissible values for the `exchange` reactions in the bounds, e.g., `E1`, `E2`, and `E3` in the schematic above. For example, bounding $0\leq\text{E2}\leq{0}$ says that metabolite `3` __cannot__ act as a `source` or `target`. On the other hand, setting $1\leq\text{E3}\leq{\infty}$ says that metabolite `8` is a target. 

In [66]:
findfirst(name -> name == "RE3", model.reactions) |> i -> model.bounds[i,1] = 1; # E3 is written as [] -> 8, so we need to constrain the lower bound on reverse
findfirst(name -> name == "FE3", model.reactions) |> i -> model.bounds[i,2] = 0; # E3 cannot be consumed, only produced

Next, we create and populate the `excludeset` variable. The `excludeset` holds the indexes of reactions that we want to `exclude` from a possible path. This requires the solver to try and find a path that meets the bounds, and the steady-state constraint, __wihtout__ using the reactions stored in the `excludeset`.

In [10]:
excludeset = Set{Int64}()
findfirst(name -> name == "R4", model.reactions) |> i-> push!(excludeset,i);

Finally, we pass the `MyStoichiometricMatrixModel` instance and the `excludeset` to the `solve(...)` method encoded in [Solve.jl](src/Solve.jl). This method reformulates the data in the `model` instance into the format required by the [GLPK.jl](https://github.com/jump-dev/GLPK.jl) linear programming solver solves the optimization problem. 
* If the problem is not feasible, e.g., we are asking the network to produce a metabolite that can't be produced given the bounds and constraints, then an error is thrown. Otherwise, a solution dictionary is returned. This dictionary has two fields, the `argmin` field is the flux vector, i.e., the min cost $\mathbf{v}$, and the `objective_value` field holds the cost of the solution, i.e., the number of reactions used to produce a `target` molecule. 

In [11]:
soln = solve(model, exclude = excludeset);

In [12]:
[model.reactions soln["argmin"]]

18×2 Matrix{Any}:
 "FE1"  0.0
 "RE1"  0.0
 "FE2"  1.0
 "RE2"  0.0
 "FE3"  0.0
 "RE3"  1.0
 "R1"   0.0
 "R2"   0.0
 "R3"   1.0
 "R4"   0.0
 "FR5"  1.0
 "RR5"  0.0
 "FR6"  1.0
 "RR6"  0.0
 "FR7"  1.0
 "RR7"  0.0
 "FR8"  1.0
 "RR8"  0.0